This file, once complete, will obtain a dataframe that contains every trade that occured during the 2019-20 NHL season. I am going to scrape the CapFriendly Website using beautiful soup and selenium and then use pandas to get it into the dataframe.

In [2]:
import requests
import pandas as pd
import time
import logging
import lxml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Now that necessary packages are installed, going to parse the needed url. This URL has a dynamic table that contains every trade that occured in the league for the 2015-16 season. So I am going to scrape the cap friendly website (Trades tab) and get this into a dataframe that I can eventually download and put into the MySQL database. As mentioned, since the table is dynamic and not static, the selenium package is needed so that after the load more button is clicked and no more rows appear (once the page is opened in the selenium server), the table is then downloaded to a dataframe.

In [4]:
# Set up the Selenium driver (make sure you have the correct path to your WebDriver)
driver_path = 'C:/STL Blues Analytics/SalaryCap/NHLSalaryCap/chromedriver.exe'  # Update this to your WebDriver path
service= Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.capfriendly.com/trades?season=2016"
driver.get(url)

# Wait for the page to load and click "Load More" until all rows are loaded
while True:
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            #EC.element_to_be_clickable((By.XPATH, "//button[text()='Load More']"))
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[10]/div/div/div[21]/div[1]"))
        )
        load_more_button.click()
        time.sleep(5)  # wait for the new rows to load
        # Check if "No results found" text is present
        no_results_text = driver.find_elements(By.XPATH, "/html/body/div[10]/div/div/div[20]/table/tbody/tr[154]/td/div")
        if no_results_text:
            break  # if "No results found" text is present, exit the loop
    except:
        break  # if no more "Load More" button, exit the loop

# Get the page source after all rows are loaded
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table and convert it to a DataFrame
table = soup.find('table')
dfattempt = pd.read_html(str(table))[0]


C:\Users\nsofi\AppData\Local\Temp\ipykernel_28300\2498100094.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfattempt = pd.read_html(str(table))[0]


In [5]:
#print(dfattempt)
dfattempt.to_csv("Sel2016Att1.csv")

In [6]:
dfattempt.shape
dfattempt.tail(5)

,DATE,TRADE DETAILS,TRADE DETAILS.1,TRADE DETAILS.2,TRADE DETAILS.3,TRADE DETAILS.4,TRADE DETAILS.5,TRADE DETAILS.6,TRADE DETAILS.7,TRADE DETAILS.8,...,TRADE DETAILS.24,TRADE DETAILS.25,TRADE DETAILS.26,TRADE DETAILS.27,TRADE DETAILS.28,TRADE DETAILS.29,TRADE DETAILS.30,TRADE DETAILS.31,TRADE DETAILS.32,TRADE DETAILS.33
100,"Jul. 1, 2015",Nashville Predators Acquire:2017 6th round pic...,Nashville Predators Acquire:2017 6th round pic...,Nashville Predators Acquire:2017 6th round pic...,Nashville Predators Acquire:2017 6th round pic...,Nashville Predators Acquire:2017 6th round pic...,Nashville Predators Acquire:2017 6th round pic...,Nashville Predators Acquire:2017 6th round pic...,Nashville Predators Acquire:2017 6th round pic...,Nashville Predators Acquire:2017 6th round pic...,...,New York Rangers Acquire:Magnus Hellberg · $62...,New York Rangers Acquire:Magnus Hellberg · $62...,New York Rangers Acquire:Magnus Hellberg · $62...,New York Rangers Acquire:Magnus Hellberg · $62...,New York Rangers Acquire:Magnus Hellberg · $62...,New York Rangers Acquire:Magnus Hellberg · $62...,New York Rangers Acquire:Magnus Hellberg · $62...,New York Rangers Acquire:Magnus Hellberg · $62...,New York Rangers Acquire:Magnus Hellberg · $62...,New York Rangers Acquire:Magnus Hellberg · $62...
101,"Jul. 1, 2015",Calgary Flames Acquire:2016 4th round pick (NS...,Calgary Flames Acquire:2016 4th round pick (NS...,Calgary Flames Acquire:2016 4th round pick (NS...,Calgary Flames Acquire:2016 4th round pick (NS...,Calgary Flames Acquire:2016 4th round pick (NS...,Calgary Flames Acquire:2016 4th round pick (NS...,Calgary Flames Acquire:2016 4th round pick (NS...,Calgary Flames Acquire:2016 4th round pick (NS...,Calgary Flames Acquire:2016 4th round pick (NS...,...,Nashville Predators Acquire:Max Reinhart · $0$...,Nashville Predators Acquire:Max Reinhart · $0$...,Nashville Predators Acquire:Max Reinhart · $0$...,Nashville Predators Acquire:Max Reinhart · $0$...,Nashville Predators Acquire:Max Reinhart · $0$...,Nashville Predators Acquire:Max Reinhart · $0$...,Nashville Predators Acquire:Max Reinhart · $0$...,Nashville Predators Acquire:Max Reinhart · $0$...,Nashville Predators Acquire:Max Reinhart · $0$...,Nashville Predators Acquire:Max Reinhart · $0$...
102,"Jul. 1, 2015","Montreal Canadiens Acquire:Zack Kassian · $1,7...","Montreal Canadiens Acquire:Zack Kassian · $1,7...","Montreal Canadiens Acquire:Zack Kassian · $1,7...","Montreal Canadiens Acquire:Zack Kassian · $1,7...","Montreal Canadiens Acquire:Zack Kassian · $1,7...","Montreal Canadiens Acquire:Zack Kassian · $1,7...","Montreal Canadiens Acquire:Zack Kassian · $1,7...","Montreal Canadiens Acquire:Zack Kassian · $1,7...","Montreal Canadiens Acquire:Zack Kassian · $1,7...",...,"Vancouver Canucks Acquire:Brandon Prust · $2,5...","Vancouver Canucks Acquire:Brandon Prust · $2,5...","Vancouver Canucks Acquire:Brandon Prust · $2,5...","Vancouver Canucks Acquire:Brandon Prust · $2,5...","Vancouver Canucks Acquire:Brandon Prust · $2,5...","Vancouver Canucks Acquire:Brandon Prust · $2,5...","Vancouver Canucks Acquire:Brandon Prust · $2,5...","Vancouver Canucks Acquire:Brandon Prust · $2,5...","Vancouver Canucks Acquire:Brandon Prust · $2,5...","Vancouver Canucks Acquire:Brandon Prust · $2,5..."
103,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,...,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,NaN
104,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,...,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,No Results Found,NaN


BOOM!! Scraped all 79 rows I need dynamically using the selenium driver. Now do the data cleaning that I have worked on and come close to getting fully corrected on it. From there, make the last fixes manually. About 90% of getting a cleaned trades dataframe is automated with 10% being manual inside the csv file itslef.

In [7]:
dfattempt.head(5)

,DATE,TRADE DETAILS,TRADE DETAILS.1,TRADE DETAILS.2,TRADE DETAILS.3,TRADE DETAILS.4,TRADE DETAILS.5,TRADE DETAILS.6,TRADE DETAILS.7,TRADE DETAILS.8,...,TRADE DETAILS.24,TRADE DETAILS.25,TRADE DETAILS.26,TRADE DETAILS.27,TRADE DETAILS.28,TRADE DETAILS.29,TRADE DETAILS.30,TRADE DETAILS.31,TRADE DETAILS.32,TRADE DETAILS.33
0,"Jun. 29, 2016","Montreal Canadiens Acquire:Shea Weber · $7,857...","Montreal Canadiens Acquire:Shea Weber · $7,857...","Montreal Canadiens Acquire:Shea Weber · $7,857...","Montreal Canadiens Acquire:Shea Weber · $7,857...","Montreal Canadiens Acquire:Shea Weber · $7,857...","Montreal Canadiens Acquire:Shea Weber · $7,857...","Montreal Canadiens Acquire:Shea Weber · $7,857...","Montreal Canadiens Acquire:Shea Weber · $7,857...","Montreal Canadiens Acquire:Shea Weber · $7,857...",...,"Nashville Predators Acquire:P.K. Subban · $9,0...","Nashville Predators Acquire:P.K. Subban · $9,0...","Nashville Predators Acquire:P.K. Subban · $9,0...","Nashville Predators Acquire:P.K. Subban · $9,0...","Nashville Predators Acquire:P.K. Subban · $9,0...","Nashville Predators Acquire:P.K. Subban · $9,0...","Nashville Predators Acquire:P.K. Subban · $9,0...","Nashville Predators Acquire:P.K. Subban · $9,0...","Nashville Predators Acquire:P.K. Subban · $9,0...","Nashville Predators Acquire:P.K. Subban · $9,0..."
1,"Jun. 29, 2016","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","Edmonton Oilers Acquire:Adam Larsson · $4,166,...",...,"New Jersey Devils Acquire:Taylor Hall · $6,000...","New Jersey Devils Acquire:Taylor Hall · $6,000...","New Jersey Devils Acquire:Taylor Hall · $6,000...","New Jersey Devils Acquire:Taylor Hall · $6,000...","New Jersey Devils Acquire:Taylor Hall · $6,000...","New Jersey Devils Acquire:Taylor Hall · $6,000...","New Jersey Devils Acquire:Taylor Hall · $6,000...","New Jersey Devils Acquire:Taylor Hall · $6,000...","New Jersey Devils Acquire:Taylor Hall · $6,000...","New Jersey Devils Acquire:Taylor Hall · $6,000..."
2,"Jun. 27, 2016","Calgary Flames Acquire:Alex Chiasson · $800,00...","Calgary Flames Acquire:Alex Chiasson · $800,00...","Calgary Flames Acquire:Alex Chiasson · $800,00...","Calgary Flames Acquire:Alex Chiasson · $800,00...","Calgary Flames Acquire:Alex Chiasson · $800,00...","Calgary Flames Acquire:Alex Chiasson · $800,00...","Calgary Flames Acquire:Alex Chiasson · $800,00...","Calgary Flames Acquire:Alex Chiasson · $800,00...","Calgary Flames Acquire:Alex Chiasson · $800,00...",...,Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Ottawa Senators Acquire:Patrick Sieloff · $0$8...
3,"Jun. 25, 2016",Arizona Coyotes Acquire:Anthony Deangelo · $86...,Arizona Coyotes Acquire:Anthony Deangelo · $86...,Arizona Coyotes Acquire:Anthony Deangelo · $86...,Arizona Coyotes Acquire:Anthony Deangelo · $86...,Arizona Coyotes Acquire:Anthony Deangelo · $86...,Arizona Coyotes Acquire:Anthony Deangelo · $86...,Arizona Coyotes Acquire:Anthony Deangelo · $86...,Arizona Coyotes Acquire:Anthony Deangelo · $86...,Arizona Coyotes Acquire:Anthony Deangelo · $86...,...,Tampa Bay Lightning Acquire:2016 2nd round pic...,Tampa Bay Lightning Acquire:2016 2nd round pic...,Tampa Bay Lightning Acquire:2016 2nd round pic...,Tampa Bay Lightning Acquire:2016 2nd round pic

In [6]:
df= pd.read_csv("Sel2016Att1.csv", header=1)
print(df)

     Unnamed: 0           DATE  \
0             0  Jun. 30, 2017   
1             1  Jun. 29, 2017   
2             2  Jun. 26, 2017   
3             3  Jun. 24, 2017   
4             4  Jun. 24, 2017   
..          ...            ...   
101         101   Oct. 7, 2016   
102         102  Aug. 25, 2016   
103         103  Jul. 18, 2016   
104         104   Jul. 8, 2016   
105         105   Jul. 2, 2016   

                                         TRADE DETAILS  \
0    Buffalo Sabres Acquire:Jason Pominville · $5,6...   
1    Calgary Flames Acquire:Eddie Läck ($1,375,000 ...   
2    Dallas Stars Acquire:Marc Methot · $4,900,000$...   
3    Calgary Flames Acquire:Travis Hamonic · $3,857...   
4    Arizona Coyotes Acquire:2017 3rd round pick (S...   
..                                                 ...   
101  Edmonton Oilers Acquire:Zachary Pochiro · $0$6...   
102  Arizona Coyotes Acquire:Dave Bolland · $5,500,...   
103  New York Rangers Acquire:Mika Zibanejad · $2,6...   
104  Anahei

In [7]:
df.head(5)

,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.1,TRADE DETAILS.2,TRADE DETAILS.3,TRADE DETAILS.4,TRADE DETAILS.5,TRADE DETAILS.6,TRADE DETAILS.7,...,TRADE DETAILS.24,TRADE DETAILS.25,TRADE DETAILS.26,TRADE DETAILS.27,TRADE DETAILS.28,TRADE DETAILS.29,TRADE DETAILS.30,TRADE DETAILS.31,TRADE DETAILS.32,TRADE DETAILS.33
0,0,"Jun. 30, 2017","Buffalo Sabres Acquire:Jason Pominville · $5,6...","Buffalo Sabres Acquire:Jason Pominville · $5,6...","Buffalo Sabres Acquire:Jason Pominville · $5,6...","Buffalo Sabres Acquire:Jason Pominville · $5,6...","Buffalo Sabres Acquire:Jason Pominville · $5,6...","Buffalo Sabres Acquire:Jason Pominville · $5,6...","Buffalo Sabres Acquire:Jason Pominville · $5,6...","Buffalo Sabres Acquire:Jason Pominville · $5,6...",...,"Minnesota Wild Acquire:Tyler Ennis · $0$0$3,65...","Minnesota Wild Acquire:Tyler Ennis · $0$0$3,65...","Minnesota Wild Acquire:Tyler Ennis · $0$0$3,65...","Minnesota Wild Acquire:Tyler Ennis · $0$0$3,65...","Minnesota Wild Acquire:Tyler Ennis · $0$0$3,65...","Minnesota Wild Acquire:Tyler Ennis · $0$0$3,65...","Minnesota Wild Acquire:Tyler Ennis · $0$0$3,65...","Minnesota Wild Acquire:Tyler Ennis · $0$0$3,65...","Minnesota Wild Acquire:Tyler Ennis · $0$0$3,65...","Minnesota Wild Acquire:Tyler Ennis · $0$0$3,65..."
1,1,"Jun. 29, 2017","Calgary Flames Acquire:Eddie Läck ($1,375,000 ...","Calgary Flames Acquire:Eddie Läck ($1,375,000 ...","Calgary Flames Acquire:Eddie Läck ($1,375,000 ...","Calgary Flames Acquire:Eddie Läck ($1,375,000 ...","Calgary Flames Acquire:Eddie Läck ($1,375,000 ...","Calgary Flames Acquire:Eddie Läck ($1,375,000 ...","Calgary Flames Acquire:Eddie Läck ($1,375,000 ...","Calgary Flames Acquire:Eddie Läck ($1,375,000 ...",...,Carolina Hurricanes Acquire:Keegan Kanzig · $0...,Carolina Hurricanes Acquire:Keegan Kanzig · $0...,Carolina Hurricanes Acquire:Keegan Kanzig · $0...,Carolina Hurricanes Acquire:Keegan Kanzig · $0...,Carolina Hurricanes Acquire:Keegan Kanzig · $0...,Carolina Hurricanes Acquire:Keegan Kanzig · $0...,Carolina Hurricanes Acquire:Keegan Kanzig · $0...,Carolina Hurricanes Acquire:Keegan Kanzig · $0...,Carolina Hurricanes Acquire:Keegan Kanzig · $0...,Carolina Hurricanes Acquire:Keegan Kanzig · $0...
2,2,"Jun. 26, 2017","Dallas Stars Acquire:Marc Methot · $4,900,000$...","Dallas Stars Acquire:Marc Methot · $4,900,000$...","Dallas Stars Acquire:Marc Methot · $4,900,000$...","Dallas Stars Acquire:Marc Methot · $4,900,000$...","Dallas Stars Acquire:Marc Methot · $4,900,000$...","Dallas Stars Acquire:Marc Methot · $4,900,000$...","Dallas Stars Acquire:Marc Methot · $4,900,000$...","Dallas Stars Acquire:Marc Methot · $4,900,000$...",...,Vegas Golden Knights Acquire:Dylan Ferguson · ...,Vegas Golden Knights Acquire:Dylan Ferguson · ...,Vegas Golden Knights Acquire:Dylan Ferguson · ...,Vegas Golden Knights Acquire:Dylan Ferguson · ...,Vegas Golden Knights Acquire:Dylan Ferguson · ...,Vegas Golden Knights Acquire:Dylan Ferguson · ...,Vegas Golden Knights Acquire:Dylan Ferguson · ...,Vegas Golden Knights Acquire:Dylan Ferguson · ...,Vegas Golden Knights Acquire:Dylan Ferguson · ...,Vegas Golden Knights Acquire:Dylan Ferguson · ...
3,3,"Jun. 24, 2017","Calgary Flames Acquire:Travis Hamonic · $3,857...","Calgary Flames Acquire:Travis Hamonic · $3,857...","Calgary Flames Acquire:Travis Hamonic · $3,857...","Calgary Flames Acquire:Travis Hamonic · $3,857...","Calgary Flames Acquire:Travis Hamonic · $3,857...","Calgary Flames Acquire:Travis Hamonic · $3,857...","Calgary Flames Acquire:Travis Hamonic · $3,857...","Calgary Flames Acquire:Travis Hamonic · $3,857...",...,New York Islanders Acquire:2018 1st round pick...,New York Islanders Acquire:2018 1st round pick...,New York Islanders Acquire:2018 1st round pick...,New York Islanders Acquire:2018 1st round pick...,New York Islanders Acquire:2018 1st round pick...,New York Islanders Acquire:2018 1st round pick...,New York Islanders Acquire:2018 1st round pick...,New York Islanders Acquire:2018 1st round pick...,New

Alright so I can see that everthing I need got extracted into the dataframe which is awesome. All the information I need can get successfully pulled. However, the format is an absolute disaster. I need to filter what information is going into the df and how it is getting pulled. Just going to send to a csv file real quick to view it better.

Alright from looking at it, A ton of information gets repeated (Lot of repeat variables). SO, I just need to filter and then use some text splitting to get information where it needs to go in the dataframe. I am going to attempt to drop the repetive columns of the dataframe in the next code chunk. Since I am dropping all but two columns, I am simply going to make a new dataframe that contains two columns.

In [8]:
better_df= dfattempt[['DATE','TRADE DETAILS', 'TRADE DETAILS.18']]
better_df.head(5)
better_df.to_csv('All Correct Info2015-16.csv')

Now that all the repetitive columns are gone, I am going to split the good trade details column into multiple columns to make the dataframe much cleaner and easier to read. 

In [9]:
better_df[['Team_x', 'x_receieved']] = better_df['TRADE DETAILS'].str.split('Acquire:', expand=True)
better_df[['Team_y', 'y_receieved']] = better_df['TRADE DETAILS.18'].str.split('Acquire:', expand=True)
better_df.head(5)
better_df.to_csv('Close2015-16.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_28300\994555388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  better_df[['Team_x', 'x_receieved']] = better_df['TRADE DETAILS'].str.split('Acquire:', expand=True)
C:\Users\nsofi\AppData\Local\Temp\ipykernel_28300\994555388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  better_df[['Team_x', 'x_receieved']] = better_df['TRADE DETAILS'].str.split('Acquire:', expand=True)
C:\Users\nsofi\AppData\Local\Temp\ipykernel_28300\994555388.py:2: SettingWithCopyWarni

In [10]:
#Have to re-read the Close.csv file back in without the last two rows which have nonsense in them. Making the edit straight in csv file and then just re-reading the csv file after
better_df=pd.read_csv("Close2015-16.csv")

In [11]:
# Define the function to split based on conditions
def conditional_splitx(x_receieved):
    if x_receieved.startswith('20') and "[Conditional]" not in x_receieved:
        parts = x_receieved.rsplit(')', 1)
        receivedx = parts[0] + ')'
        Monetary = parts[1] if len(parts) > 1 else ''
    elif x_receieved.startswith('20') and "[Conditional]" in x_receieved:
        parts = x_receieved.rsplit('Sum', 1)
        receivedx = parts[0] 
        Monetary = parts[1] if len(parts) > 1 else ''
    elif "Future Considerations" in x_receieved:
        parts =x_receieved.rsplit('Sum', 1)
        receivedx = parts[0] 
        Monetary = parts[1] if len(parts) > 1 else ''
    else:
        parts = x_receieved.split(' · ', 1)
        receivedx = parts[0]
        Monetary = ' · ' + parts[1] if len(parts) > 1 else ''
    return pd.Series([receivedx, Monetary])

# Apply the function and create new columns
better_df[['receivedx', 'Monetary']] = better_df['x_receieved'].apply(conditional_splitx)
#better_df[['X_received']] = better_df['receivedx'].str.split('Sum',expand=True)




better_df.head(5)
#better_df.to_csv('Progress2022-23.csv')


,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.18,Team_x,x_receieved,Team_y,y_receieved,receivedx,Monetary
0,0,"Jun. 29, 2016","Montreal Canadiens Acquire:Shea Weber · $7,857...","Nashville Predators Acquire:P.K. Subban · $9,0...",Montreal Canadiens,"Shea Weber · $7,857,143$7,857,143$12,000,000Su...",Nashville Predators,"P.K. Subban · $9,000,000$9,000,000$11,000,000S...",Shea Weber,"· $7,857,143$7,857,143$12,000,000Sum: $7,857,..."
1,1,"Jun. 29, 2016","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","New Jersey Devils Acquire:Taylor Hall · $6,000...",Edmonton Oilers,"Adam Larsson · $4,166,666$4,166,666$3,000,000S...",New Jersey Devils,"Taylor Hall · $6,000,000$6,000,000$6,000,000Su...",Adam Larsson,"· $4,166,666$4,166,666$3,000,000Sum: $4,166,6..."
2,2,"Jun. 27, 2016","Calgary Flames Acquire:Alex Chiasson · $800,00...",Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Calgary Flames,"Alex Chiasson · $800,000$800,000$800,000Sum: $...",Ottawa Senators,"Patrick Sieloff · $0$894,167$70,000 (AHL/JR)Su...",Alex Chiasson,"· $800,000$800,000$800,000Sum: $800,000$800,0..."
3,3,"Jun. 25, 2016",Arizona Coyotes Acquire:Anthony Deangelo · $86...,Tampa Bay Lightning Acquire:2016 2nd round pic...,Arizona Coyotes,"Anthony Deangelo · $863,333$863,333$925,000Sum...",Tampa Bay Lightning,2016 2nd round pick (ARI - #37 - Libor Hájek)S...,Anthony Deangelo,"· $863,333$863,333$925,000Sum: $863,333$863,3..."
4,4,"Jun. 25, 2016",Colorado Avalanche Acquire:2017 4th round pick...,"New York Rangers Acquire:Nick Holden · $1,650,...",Colorado Avalanche,2017 4th round pick (NYR - #114 - Petr Kvaca)S...,New York Rangers,"Nick Holden · $1,650,000$1,650,000$1,650,000Su...",2017 4th round pick (NYR - #114 - Petr Kvaca),"Sum: $0$0$0Change: -$1,650,000Change: -$1,650,..."


Just did a bunch of string manipulation to x_received, now have to do it to y_received

In [12]:
# Define the function to split based on conditions
def conditional_splity(y_receieved):
    if y_receieved.startswith('20') and "[Conditional]" not in y_receieved:
        parts = y_receieved.rsplit(')', 1)
        receivedy = parts[0] + ')'
        Monetary2 = parts[1] if len(parts) > 1 else ''
    elif y_receieved.startswith('20') and "[Conditional]" in y_receieved:
        parts =y_receieved.rsplit('Sum', 1)
        receivedy = parts[0] 
        Monetary2 = parts[1] if len(parts) > 1 else ''
    elif "Future Considerations" in y_receieved:
        parts =y_receieved.rsplit('Sum', 1)
        receivedy = parts[0] 
        Monetary2 = parts[1] if len(parts) > 1 else ''
    else:
        parts = y_receieved.split(' · ', 1)
        receivedy = parts[0]
        Monetary2 = ' · ' + parts[1] if len(parts) > 1 else ''
    return pd.Series([receivedy, Monetary2])

# Apply the function and create new columns
better_df[['receivedy', 'Monetary2']] = better_df['y_receieved'].apply(conditional_splity)
#better_df[['X_received']] = better_df['receivedx'].str.split('Sum',expand=True)




better_df.head(5)
#better_df.to_csv('Progress9.csv')

,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.18,Team_x,x_receieved,Team_y,y_receieved,receivedx,Monetary,receivedy,Monetary2
0,0,"Jun. 29, 2016","Montreal Canadiens Acquire:Shea Weber · $7,857...","Nashville Predators Acquire:P.K. Subban · $9,0...",Montreal Canadiens,"Shea Weber · $7,857,143$7,857,143$12,000,000Su...",Nashville Predators,"P.K. Subban · $9,000,000$9,000,000$11,000,000S...",Shea Weber,"· $7,857,143$7,857,143$12,000,000Sum: $7,857,...",P.K. Subban,"· $9,000,000$9,000,000$11,000,000Sum: $9,000,..."
1,1,"Jun. 29, 2016","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","New Jersey Devils Acquire:Taylor Hall · $6,000...",Edmonton Oilers,"Adam Larsson · $4,166,666$4,166,666$3,000,000S...",New Jersey Devils,"Taylor Hall · $6,000,000$6,000,000$6,000,000Su...",Adam Larsson,"· $4,166,666$4,166,666$3,000,000Sum: $4,166,6...",Taylor Hall,"· $6,000,000$6,000,000$6,000,000Sum: $6,000,0..."
2,2,"Jun. 27, 2016","Calgary Flames Acquire:Alex Chiasson · $800,00...",Ottawa Senators Acquire:Patrick Sieloff · $0$8...,Calgary Flames,"Alex Chiasson · $800,000$800,000$800,000Sum: $...",Ottawa Senators,"Patrick Sieloff · $0$894,167$70,000 (AHL/JR)Su...",Alex Chiasson,"· $800,000$800,000$800,000Sum: $800,000$800,0...",Patrick Sieloff,"· $0$894,167$70,000 (AHL/JR)Sum: $0$894,167$7..."
3,3,"Jun. 25, 2016",Arizona Coyotes Acquire:Anthony Deangelo · $86...,Tampa Bay Lightning Acquire:2016 2nd round pic...,Arizona Coyotes,"Anthony Deangelo · $863,333$863,333$925,000Sum...",Tampa Bay Lightning,2016 2nd round pick (ARI - #37 - Libor Hájek)S...,Anthony Deangelo,"· $863,333$863,333$925,000Sum: $863,333$863,3...",2016 2nd round pick (ARI - #37 - Libor Hájek),"Sum: $0$0$0Change: -$863,333Change: -$863,333C..."
4,4,"Jun. 25, 2016",Colorado Avalanche Acquire:2017 4th round pick...,"New York Rangers Acquire:Nick Holden · $1,650,...",Colorado Avalanche,2017 4th round pick (NYR - #114 - Petr Kvaca)S...,New York Rangers,"Nick Holden · $1,650,000$1,650,000$1,650,000Su...",2017 4th round pick (NYR - #114 - Petr Kvaca),"Sum: $0$0$0Change: -$1,650,000Change: -$1,650,...",Nick Holden,"· $1,650,000$1,650,000$1,650,000Sum: $1,650,0..."


Alright ton of progress made, definitely not 100% correct but some manual labor is going to be needed in addition to the web scraping. But, the web scraping saves a ton of time so going to move forward with it for other seasons to build out and help finish the rest of the trade datasets for other seasons. 

To finish the coding aspect of the trades that occured during the 2022-23 season, I am going to delete the extra columns. From there, I will finish this dataset completely by manually editing some of the data in the csv file itslef. 

In [13]:
better_df.head(2)

,Unnamed: 0,DATE,TRADE DETAILS,TRADE DETAILS.18,Team_x,x_receieved,Team_y,y_receieved,receivedx,Monetary,receivedy,Monetary2
0,0,"Jun. 29, 2016","Montreal Canadiens Acquire:Shea Weber · $7,857...","Nashville Predators Acquire:P.K. Subban · $9,0...",Montreal Canadiens,"Shea Weber · $7,857,143$7,857,143$12,000,000Su...",Nashville Predators,"P.K. Subban · $9,000,000$9,000,000$11,000,000S...",Shea Weber,"· $7,857,143$7,857,143$12,000,000Sum: $7,857,...",P.K. Subban,"· $9,000,000$9,000,000$11,000,000Sum: $9,000,..."
1,1,"Jun. 29, 2016","Edmonton Oilers Acquire:Adam Larsson · $4,166,...","New Jersey Devils Acquire:Taylor Hall · $6,000...",Edmonton Oilers,"Adam Larsson · $4,166,666$4,166,666$3,000,000S...",New Jersey Devils,"Taylor Hall · $6,000,000$6,000,000$6,000,000Su...",Adam Larsson,"· $4,166,666$4,166,666$3,000,000Sum: $4,166,6...",Taylor Hall,"· $6,000,000$6,000,000$6,000,000Sum: $6,000,0..."


In [14]:
Final_df= better_df[['DATE','TRADE DETAILS', 'TRADE DETAILS.18','Team_x','receivedx','Team_y','receivedy']]


In [15]:
Final_df.to_csv("NHL 2015-16 Trade Data.csv")

Just a coding note:
To replace part of a string inside a variable of a dataset do the following: 
better_df['part1'] = better_df['part1'].str.replace(':', '')